In [7]:
import cv2
import numpy as np
import pandas as pd
from deepface import DeepFace
from datetime import datetime

# --- Input Video Path ---
VIDEO_PATH = r"C:\Users\hp\Downloads\WhatsApp Video 2025-07-30 at 3.15.38 PM.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("❌ Could not open video.")
    exit()

# --- Globals ---
known_faces = []
selected_id = None
selected_emotions = set()
MATCH_THRESHOLD = 0.35
next_id = 1
frame_counter = 0
FRAME_SKIP = 2  # Skip every 5 frames for speed

# --- Cosine Distance ---
def find_cosine_distance(a, b):
    return 1 - (np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

# --- Face Selection ---
def select_face(event, x, y, flags, param):
    global selected_id
    if event == cv2.EVENT_LBUTTONDOWN:
        for face in known_faces:
            fx, fy, fw, fh = face["region"]
            if fx <= x <= fx + fw and fy <= y <= fy + fh:
                if selected_id != face["id"]:
                    selected_id = face["id"]
                    selected_emotions.clear()
                    print(f"✅ Selected: {selected_id}")
                break

cv2.namedWindow("Video Emotion Tracker")
cv2.setMouseCallback("Video Emotion Tracker", select_face)

# --- Frame Loop ---
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_counter += 1
    if frame_counter % FRAME_SKIP != 0:
        continue

    frame = cv2.resize(frame, (480, 360))

    try:
        results = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

        if not isinstance(results, list):
            results = [results]

        current_faces = []
        for result in results:
            region = result.get("region", {})
            if not region:
                continue

            x, y, w, h = region['x'], region['y'], region['w'], region['h']
            face_img = frame[y:y+h, x:x+w]
            if face_img.size == 0:
                continue

            reps = DeepFace.represent(face_img, model_name='SFace', enforce_detection=False)
            if not reps:
                continue
            embedding = reps[0]['embedding']

            matched_id = None
            for face in known_faces:
                dist = find_cosine_distance(face['embedding'], embedding)
                if dist < MATCH_THRESHOLD:
                    matched_id = face['id']
                    face['region'] = (x, y, w, h)
                    break

            if matched_id is None:
                matched_id = f"Person_{next_id}"
                next_id += 1
                known_faces.append({
                    'id': matched_id,
                    'embedding': embedding,
                    'region': (x, y, w, h)
                })

            current_faces.append({
                'id': matched_id,
                'region': (x, y, w, h),
                'emotion': result['dominant_emotion']
            })

        # --- Draw Boxes ---
        for face in current_faces:
            pid = face["id"]
            x, y, w, h = face["region"]
            emotion = face["emotion"]
            color = (0, 255, 255) if pid == selected_id else (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

            if pid == selected_id:
                if emotion not in selected_emotions:
                    selected_emotions.add(emotion)
                label = f"{pid}: {', '.join(selected_emotions)}"
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    except Exception as e:
        print("⚠️ Error:", e)

    cv2.imshow("Video Emotion Tracker", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# --- Save Summary to CSV ---
if selected_id and selected_emotions:
    pd.DataFrame([{
        "PersonID": selected_id,
        "Emotions": ", ".join(selected_emotions),
        "Date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }]).to_csv("selected_emotion_summary.csv", index=False)
    print("✅ Saved summary to selected_emotion_summary.csv")
else:
    print("⚠️ No selection made or no emotions detected.")

✅ Selected: Person_1
✅ Saved summary to selected_emotion_summary.csv
